# TP4 - Non-negative Matrix Factorization
The goal is to study the use of nonnegative matrix factorisation (NMF) for topic extraction from a dataset of text documents. The rationale is to interpret each extracted NMF component as being associated with a specific topic. 

Study and test the following script (introduced  on [scikit](http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html))

1. Test and comment on the effect of varying the initialisation, especially using random
nonnegative values as initial guesses (for W and H coefficients, using the notations introduced
during the lecture).
2. Compare and comment on the difference between the results obtained with `2 cost compared
to the generalised Kullback-Liebler cost.
3. Test and comment on the results obtained using a simpler term-frequency representation
as input (as opposed to the TF-IDF representation considered in the code above) when
considering the Kullback-Liebler cost.

In [1]:
###### CUSTOM NMF IMPLEMENTATION ######
# Multiplicative Update Rules for NMF #
# estimation with beta divergences    #
import numpy

# TODO: translate slides 59 [beta-divergence] & 47 [error and special cases]

def custom_NMF(V, K, W=None, H=None, steps=50, beta=0, toll=0.1, show_div=False):
    
    F = len(V) #Number of V rows
    N = len(V[0]) #Number of V columns

    if W is None:
        W = numpy.random.rand(F,K)
        
    if H is None:
        H = numpy.random.rand(K,N)
        
    if N != len(H[0]):
        raise ValueError("Size for H[0] is different - found "+str(len(H[0]))+" in place of "+str(N))
    if F != len(W):
        raise ValueError("Size for F is different - found "+str(len(F))+" in place of "+str(N))
        
    #Setup n_iter
    n_iter = 1
    
    # Setup initial error
    init_error = _beta_div(V,W,H,beta,F,N,K)
    if show_div:
        print("Initial error: "+str(init_error))
    error = init_error
    
    for step in range(steps):
    
#         Tests with whole matrix : multiply = O | dot = *
        upd_UP = numpy.dot(W.T, numpy.multiply(pow(numpy.dot(W,H),beta-2), V))
        upd_DOWN = numpy.dot(W.T, pow(numpy.dot(W,H),beta-1))
        upd = upd_UP / upd_DOWN
        H = numpy.multiply(H, upd)
        
        upd_UP = numpy.dot(numpy.multiply(pow(numpy.dot(W,H),beta-2), V),H.T)
        upd_DOWN = numpy.dot(pow(numpy.dot(W,H),beta-1), H.T)
        upd = upd_UP / upd_DOWN
        W = numpy.multiply(W, upd)

        # Test element-wise products
#         for i in range(F):
#             for j in range(N):
#                 for k in range(K):
#                     x = V[i][j]
#                     w = W[i][k]
#                     h = H[k][j]
#                     y = w*h
# #                     print("x:"+str(x)+" | w:"+str(w)+" | h:"+str(h)+" | y:"+str(y))
#                     # Update h
#                     upd_up = w*(pow(y,beta-2)*x)
#                     upd_down = w*pow(y,beta-1)
#                     upd = upd_up/upd_down
#                     h = h*upd
#                     # Update w
#                     upd_up = (pow(y,beta-2)*x)*h
#                     upd_down = pow(y,beta-1)*h
#                     upd = upd_up/upd_down
#                     w = w*upd
        
        if toll > 0:
            new_error = _beta_div(V,W,H,beta,F,N,K)
            if show_div:
                print("Error on iteration "+str(n_iter)+": " +str(new_error))
            # Check if approximation error relative decrease is below the desired threshold
            if ((error - new_error) / init_error) < toll:
                break
            error = new_error
            
        n_iter += 1
            
    return W, H

def _beta_div(V,W,H,beta,F,N,K):
    div = 0
    # Update beta_divergence
    if beta == 1: # generalized Kullback-Leibler divergence. x log(x/y) - x + y
        # div = numpy.dot(V, numpy.log(V,numpy.dot(W,H))) - numpy.sum(V) + numpy.sum(numpy.dot(W,H))
        for i in range(F):
            for j in range(N):
                for k in range(K):
                    x = V[i][j]
                    y = W[i][k]*H[k][j]
                    div += x*numpy.log(x/y) - x + y
    elif beta == 0: # Itakura-Saito divergence. (x/y) - log(x/y) -1
        # div = numpy.sum(V / numpy.dot(W,H)) - numpy.sum(numpy.log(V / numpy.dot(W,H))) - numpy.product(len(V))
        for i in range(F):
            for j in range(N):
                for k in range(K):
                    x = V[i][j]
                    y = W[i][k]*H[k][j]
                    div += (x/y) * numpy.log(x/y) - 1
    else: # Euclidean distance. (1/beta(beta-1))(x^beta + (beta-1)y^beta - beta*x*y^beta-1)
        left_factor = 1/(beta*(beta-1))
        # div = left_factor*(pow(numpy.sum(V), beta) + (beta-1)*pow(numpy.sum(numpy.dot(W,H),beta)) - beta*numpy.sum(V)*pow(numpy.sum(numpy.dot(W,H),beta)))
        for i in range(F):
            for j in range(N):
                for k in range(K):
                    x = V[i][j]
                    y = W[i][k]*H[k][j]
                    div += left_factor*(pow(x,beta) + (beta-1)*pow(y,beta) - beta*x*pow(y,beta-1))
    return div

#######

if __name__ == "__main__":
    V = [
         [5,3,0,1],
         [4,0,0,1],
         [1,1,0,5],
         [1,0,0,4],
         [0,1,5,4],
        ]

    V = numpy.array(V) # Data matrix F x N 
    K = 2

    W, H = custom_NMF(V, K, beta = 2, toll = 0.0001, show_div = True)

Initial error: 126.87913176003777
Error on iteration 1: 83.05725059261255
Error on iteration 2: 78.03751930502544
Error on iteration 3: 75.73357179188885
Error on iteration 4: 75.42456719235722
Error on iteration 5: 75.27507869109539
Error on iteration 6: 74.89451925327508
Error on iteration 7: 74.58897173405492
Error on iteration 8: 74.5004458586124
Error on iteration 9: 74.59616319908983


In [2]:
##### TEST RESULTS #####
W

array([[0.00404669, 1.11801199],
       [0.00189289, 0.75267693],
       [1.01006549, 0.39385685],
       [0.77534413, 0.31250428],
       [1.7904365 , 0.03975826]])